In [27]:
import os
import requests
import datetime
# import calendar

import pandas as pd
from dotenv import load_dotenv
from sqlmodel import SQLModel, create_engine
from sqlalchemy.orm import sessionmaker

from heatmap.conversion import ng_to_core_power_carbon_intensity
from heatmap.definitions.ng_carbon_intensity import Response as NgCarbonIntensityResponse
from heatmap.definitions.timeseries import PowerCarbonIntensity, PowerPrice, GasPrice

In [2]:
load_dotenv('../.env')
DB_CONN_STR = os.environ['DB_CONN_STR'].replace('sqlite:///', 'sqlite:///../')

engine = create_engine(url=DB_CONN_STR)
SQLModel.metadata.create_all(engine, tables=[
    PowerCarbonIntensity.__table__, 
    PowerPrice.__table__, 
    GasPrice.__table__
])

In [23]:
def batch_months(start_date, end_date):
    current = start_date

    batches = []

    while current <= end_date:
        month_end = datetime.datetime(current.year, current.month+1, 1, 0, 0)
        
        if month_end > end_date:
            month_end = end_date

        batches.append((current, month_end))

        if month_end == end_date:
            break
        current = month_end + datetime.timedelta(days=1)
    
    return batches

In [31]:
start_date = datetime.datetime(2021, 1, 1, 0, 0)
end_date = datetime.datetime(2021, 3, 30, 23, 59)

# month_start_end_dates = batch_months(start_date, end_date)

dt_rng = pd.date_range(start_date, end_date, freq='7d')

list(zip(dt_rng[1:], dt_rng[:-1]))

[(Timestamp('2021-01-08 00:00:00'), Timestamp('2021-01-01 00:00:00')),
 (Timestamp('2021-01-15 00:00:00'), Timestamp('2021-01-08 00:00:00')),
 (Timestamp('2021-01-22 00:00:00'), Timestamp('2021-01-15 00:00:00')),
 (Timestamp('2021-01-29 00:00:00'), Timestamp('2021-01-22 00:00:00')),
 (Timestamp('2021-02-05 00:00:00'), Timestamp('2021-01-29 00:00:00')),
 (Timestamp('2021-02-12 00:00:00'), Timestamp('2021-02-05 00:00:00')),
 (Timestamp('2021-02-19 00:00:00'), Timestamp('2021-02-12 00:00:00')),
 (Timestamp('2021-02-26 00:00:00'), Timestamp('2021-02-19 00:00:00')),
 (Timestamp('2021-03-05 00:00:00'), Timestamp('2021-02-26 00:00:00')),
 (Timestamp('2021-03-12 00:00:00'), Timestamp('2021-03-05 00:00:00')),
 (Timestamp('2021-03-19 00:00:00'), Timestamp('2021-03-12 00:00:00')),
 (Timestamp('2021-03-26 00:00:00'), Timestamp('2021-03-19 00:00:00'))]

In [ ]:
dt_rng[1:]

In [25]:
start_date, end_date = month_start_end_dates[0]

r = requests.get(f'https://api.carbonintensity.org.uk/intensity/{start_date.strftime("%Y-%m-%dT%H:%MZ")}/{end_date.strftime("%Y-%m-%dT%H:%MZ")}', headers={'Accept': 'application/json'})
 
ng_carbon_intensity_response = NgCarbonIntensityResponse.model_validate(r.json())
power_carbon_intensities = ng_to_core_power_carbon_intensity(ng_carbon_intensity_response)

print(len(power_carbon_intensities))
power_carbon_intensities[:5]

ValidationError: 1 validation error for Response
data
  Field required [type=missing, input_value={'error': {'code': '400 B...a smaller date range.'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [22]:
power_carbon_intensities[-5:]

[PowerCarbonIntensity(start_time=datetime.datetime(2021, 1, 31, 21, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 1, 31, 21, 30, tzinfo=TzInfo(UTC)), value=196.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 1, 31, 21, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 1, 31, 22, 0, tzinfo=TzInfo(UTC)), value=191.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 1, 31, 22, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 1, 31, 22, 30, tzinfo=TzInfo(UTC)), value=185.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 1, 31, 22, 30, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 1, 31, 23, 0, tzinfo=TzInfo(UTC)), value=182.0),
 PowerCarbonIntensity(start_time=datetime.datetime(2021, 1, 31, 23, 0, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2021, 1, 31, 23, 30, tzinfo=TzInfo(UTC)), value=182.0)]

In [ ]:
bound_sessionmaker = sessionmaker(engine)

with bound_sessionmaker() as session:
    for power_carbon_intensity in power_carbon_intensities:
        session.add(power_carbon_intensity)
    session.commit()